In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from scipy.io import mmread

In [27]:
mmread('1138_bus/1138_bus.mtx')

<1138x1138 sparse matrix of type '<class 'numpy.float64'>'
	with 4054 stored elements in COOrdinate format>

In [2]:
from lib.imports import *
from lib.functions import *
from lib.utils import *
from lib.modules import *
from lib.data import *
from ipynb.fs.defs.losses import *

In [86]:
G_list = load_G_list(data_path='data/rome', index_file='data_index.txt', cache='G_list.pickle')

In [3]:
G_list_large = pickle.load(open('G_list_large.pickle', 'rb'))

In [91]:
data_list = generate_data_list(G_list, 
                               sparse=5, 
                               pivot_mode='maxmin',
                               init_mode='random',
                               edge_index='full_edge_index',
                               edge_attr='full_edge_attr',
                               pmds_list=pickle.load(open('pmds_list.pickle', 'rb')),
                               device='cpu')

preprocess G:   0%|          | 0/11531 [00:00<?, ?it/s]

In [5]:
model = Model()

In [7]:
model(batch)

tensor([[ 0.0646,  0.6262],
        [ 0.0723,  0.5733],
        [-0.1221,  1.2808],
        ...,
        [ 0.2123,  0.9736],
        [ 0.0312,  1.2881],
        [ 0.0512,  0.5859]], grad_fn=<AddBackward0>)

In [62]:
rc_x = 4000
rc_n = 200
rc_k = int(np.ceil((2 * rc_n - 1 - np.sqrt((2 * rc_n - 1) ** 2 - 4 * rc_x)) / 2))

In [63]:
rc_k * (rc_n-1) + rc_k * (rc_n-rc_k)

4268

In [65]:
G.edges

EdgeView([(0, 5), (0, 6), (0, 8), (0, 15), (1, 12), (2, 6), (2, 15), (3, 8), (3, 10), (3, 14), (3, 15), (4, 8), (4, 11), (7, 14), (9, 11), (9, 12), (9, 15), (10, 13), (11, 13), (12, 13), (14, 15)])

In [96]:
batch = Batch.from_data_list(data_list)

In [17]:
batch.cluster_edge_sparsity.mean()

tensor(0.3503)

In [15]:
batch.grouped_edge_sparsity.mean()

tensor(0.4574)

In [24]:
batch.sparse_edge_sparsity.mean()

tensor(0.3338)

In [82]:
batch.hierarchical_sparse_edge_sparsity.mean()

tensor(0.3093)

In [9]:
batch.recursive_sparse_edge_sparsity.mean()

tensor(0.3768)

In [101]:
batch.recursive_cluster_edge_sparsity.mean()

tensor(0.1649)

In [102]:
batch.sparse_edge_sparsity_rc_k.mean()

tensor(0.1995)

In [94]:
data_list[8473]

Data(cluster_edge_attr_nbhd=[2846, 2], cluster_edge_attr_pivot=[2846, 2], cluster_edge_attr_reg=[2846, 2], cluster_edge_index=[2, 2846], cluster_edge_sparsity=0.2509257626520896, edge_attr=[11342, 2], edge_index=[2, 11342], full_edge_attr=[11342, 2], full_edge_index=[2, 11342], hierarchical_sparse_edge_attr_nbhd=[1360, 2], hierarchical_sparse_edge_attr_pivot=[1360, 2], hierarchical_sparse_edge_attr_reg=[1360, 2], hierarchical_sparse_edge_attr_sympivot=[1360, 2], hierarchical_sparse_edge_index=[2, 1360], hierarchical_sparse_edge_sparsity=0.11990830541350732, pos=[107, 2], recursive_cluster_edge_attr_nbhd=[806, 2], recursive_cluster_edge_attr_pivot=[806, 2], recursive_cluster_edge_attr_reg=[806, 2], recursive_cluster_edge_attr_sympivot=[806, 2], recursive_cluster_edge_index=[2, 806], recursive_cluster_edge_sparsity=0.0710633045318286, sparse_edge_attr_nbhd=[1300, 2], sparse_edge_attr_nbhd_hs_k=[1494, 2], sparse_edge_attr_nbhd_rc_k=[894, 2], sparse_edge_attr_pivot=[1300, 2], sparse_edge_a

In [98]:
batch.sparse_edge_sparsity

tensor([0.6583, 0.3302, 0.1925,  ..., 0.1328, 0.1787, 0.3429])

In [29]:
def generate_apsp(G):
    apsp_dict = dict(nx.all_pairs_shortest_path_length(G))
    return np.array([[apsp_dict[j][k] for k in sorted(apsp_dict[j].keys())] for j in sorted(apsp_dict.keys())])

In [58]:
def get_neighborhood_size(apsp):
    return np.cumsum(np.apply_along_axis(lambda x: np.bincount(x, minlength=len(apsp)), 1, apsp), axis=1)

In [31]:
def generate_maxmin_pivots_new(nodes, apsp, k):
    partial_apsp = apsp[nodes, :][:, nodes]
    pivots = [partial_apsp.max(axis=1).argmax()]
    for _ in range(k - 1):
        pivots.append(partial_apsp[:, pivots].min(axis=1).argmax())
    return np.array(nodes)[pivots]

In [33]:
def get_pivot_groups_new(nodes, apsp, pivots):
    partial_apsp = apsp[pivots, :][:, nodes]
    groups = np.zeros_like(partial_apsp)
    groups[partial_apsp.argmin(axis=0), np.arange(groups.shape[1])] = 1
#     return {p: (n := np.array(nodes)[g.astype(bool)])[n != p] for p, g in zip(pivots, groups)}
    return {p: np.array(nodes)[g.astype(bool)][np.array(nodes)[g.astype(bool)] != p] for p, g in zip(pivots, groups)}

In [12]:
G = G_list[0]
apsp = generate_apsp(G)

In [25]:
neighborhood = get_neighborhood_size(apsp)
neighborhood

array([[ 1,  5, 10, 14, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16],
       [ 1,  2,  4,  7, 12, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16],
       [ 1,  3,  7, 13, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16],
       [ 1,  5, 11, 15, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16],
       [ 1,  3,  7, 13, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16],
       [ 1,  2,  5, 10, 14, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16],
       [ 1,  3,  6, 10, 14, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16],
       [ 1,  2,  4,  9, 15, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16],
       [ 1,  4, 10, 14, 15, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16],
       [ 1,  4, 11, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16],
       [ 1,  3,  8, 14, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16],
       [ 1,  4,  8, 13, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16],
       [ 1,  4,  7, 12, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16],
       [ 1,  4,  8, 11, 14, 16, 16, 16, 16, 16, 16,

In [14]:
pivots = generate_maxmin_pivots_new(G.nodes, apsp, 5)
pivots

array([1, 5, 7, 2, 4])

In [18]:
get_pivot_groups(G, apsp, pivots)

{1: [1, 9, 10, 12, 13], 5: [0, 5], 7: [3, 7, 14], 2: [2, 6, 15], 4: [4, 8, 11]}

In [16]:
get_pivot_groups_new(G.nodes, apsp, pivots)

{1: array([ 9, 10, 12, 13]),
 5: array([0]),
 7: array([ 3, 14]),
 2: array([ 6, 15]),
 4: array([ 8, 11])}

In [34]:
def get_recursive_pivot_groups(nodes, apsp, k):
    pivots = generate_pivots(nodes, apsp, k)
    groups = get_pivot_groups(nodes, apsp, pivots)
    for p in pivots:
        if len(groups[p]) > k:
            groups[p] = get_recursive_pivot_groups(groups[p], apsp, k)
    return groups

In [35]:
def get_pivot_group_cardinalities(groups):
    cardin_dict = dict()
    def get_cardinalities(groups):
        if type(groups) is not dict:
            cardin_dict.update(zip(groups, np.ones_like(groups)))
            return len(groups) + 1
        cardin = list(map(get_cardinalities, groups.values()))
        cardin_dict.update(zip(groups, cardin))
        return sum(cardin) + 1
    get_cardinalities(groups)
    return np.array(sorted(cardin_dict.items()))[:, 1]

In [36]:
def get_peer_pivot_dense_edge_set(groups):
    edges = set()
    def add_all_edges(groups):
        if type(groups) is dict:
            edges.update([(p, q) for p in groups for q in groups if p != q])
            for p in groups:
                add_all_edges(groups[p])
    add_all_edges(groups)
    return edges

def get_same_level_dense_edge_set(groups):
    edges = set()
    def add_all_edges(groups):
        edges.update([(p, q) for p in groups for q in groups if p != q])
        if type(groups) is dict:
            for p in groups:
                add_all_edges(groups[p])
    add_all_edges(groups)
    return edges

def get_adjacent_level_dense_edge_set(groups):
    edges = set()
    def add_all_edges(groups):
        if type(groups) is dict:
            descendents = np.concatenate([add_all_edges(groups[p]) for p in groups])
            edges.update([(p, i) for p in groups for i in descendents])
            return np.array(list(groups))
        else:
            return groups
    add_all_edges(groups)
    return edges

def get_cross_level_dense_edge_set(groups):
    edges = set()
    def add_all_edges(groups):
        if type(groups) is dict:
            descendents = np.concatenate([add_all_edges(groups[p]) for p in groups])
            edges.update([(p, i) for p in groups for i in descendents])
            return np.concatenate((list(groups), descendents))
        else:
            return groups
    add_all_edges(groups)
    return edges

def get_tree_edge_set(groups):
    edges = set()
    def add_all_edges(groups):
        if type(groups) is dict:
            edges.update([(p, i) for p in groups for i in groups[p]])
            for p in groups:
                add_all_edges(groups[p])
    add_all_edges(groups)
    return edges

def get_forward_edge_set(groups):
    edges = set()
    def add_all_edges(groups):
        if type(groups) is dict:
            descendents = {p: add_all_edges(groups[p]) for p in groups}
            edges.update([(p, i) for p in groups for i in descendents[p]])
            return np.concatenate((list(groups), *descendents.values()))
        else:
            return groups
    add_all_edges(groups)
    return edges

In [37]:
def get_hierarchical_cluster_edge_set(groups):
    return get_tree_edge_set(groups).union(get_same_level_dense_edge_set(groups))

def get_recursive_cluster_edge_set(groups):
    return get_forward_edge_set(groups).union(get_same_level_dense_edge_set(groups))

def get_hierarchical_sparse_edge_set(groups):
    return get_adjacent_level_dense_edge_set(groups).union(get_peer_pivot_dense_edge_set(groups))

def get_recursive_sparse_edge_set(groups):
    return get_cross_level_dense_edge_set(groups).union(get_peer_pivot_dense_edge_set(groups))

In [38]:
def create_edge_index(*edge_sets, device='cpu'):
    all_edges = reduce(lambda x, y: x.union(y), edge_sets, set())
    reverse_edges = set(map(lambda x: x[::-1], all_edges))
    sorted_symmetric_edges = sorted(list(all_edges.union(reverse_edges)))
    return torch.tensor(sorted_symmetric_edges, dtype=torch.long, device=device).t()

In [39]:
def generate_regular_edge_attr_new(edge_index, apsp, device='cpu'):
    d = apsp[edge_index[0], edge_index[1]]
    w = 1 / d ** 2
    return torch.tensor(np.stack([d, w], axis=1), dtype=torch.float, device=device)

In [40]:
def generate_pivot_edge_attr_new(edge_index, apsp, cardinalities, device='cpu'):
    d = apsp[edge_index[0], edge_index[1]]
    s = cardinalities[edge_index[0]]
    w = s / d ** 2
    return torch.tensor(np.stack([d, w], axis=1), dtype=torch.float, device=device)

In [41]:
def generate_symmetric_pivot_edge_attr(edge_index, apsp, cardinalities, device='cpu'):
    d = apsp[edge_index[0], edge_index[1]]
    s = cardinalities[edge_index[0]] * cardinalities[edge_index[1]]
    w = s / d ** 2
    return torch.tensor(np.stack([d, w], axis=1), dtype=torch.float, device=device)

In [42]:
G = G_list[0]

In [43]:
apsp = generate_apsp(G)
apsp

array([[0, 4, 2, 2, 2, 1, 1, 3, 1, 2, 3, 3, 3, 4, 2, 1],
       [4, 0, 4, 4, 4, 5, 5, 5, 5, 2, 3, 3, 1, 2, 4, 3],
       [2, 4, 0, 2, 4, 3, 1, 3, 3, 2, 3, 3, 3, 4, 2, 1],
       [2, 4, 2, 0, 2, 3, 3, 2, 1, 2, 1, 3, 3, 2, 1, 1],
       [2, 4, 4, 2, 0, 3, 3, 4, 1, 2, 3, 1, 3, 2, 3, 3],
       [1, 5, 3, 3, 3, 0, 2, 4, 2, 3, 4, 4, 4, 5, 3, 2],
       [1, 5, 1, 3, 3, 2, 0, 4, 2, 3, 4, 4, 4, 5, 3, 2],
       [3, 5, 3, 2, 4, 4, 4, 0, 3, 3, 3, 4, 4, 4, 1, 2],
       [1, 5, 3, 1, 1, 2, 2, 3, 0, 3, 2, 2, 4, 3, 2, 2],
       [2, 2, 2, 2, 2, 3, 3, 3, 3, 0, 3, 1, 1, 2, 2, 1],
       [3, 3, 3, 1, 3, 4, 4, 3, 2, 3, 0, 2, 2, 1, 2, 2],
       [3, 3, 3, 3, 1, 4, 4, 4, 2, 1, 2, 0, 2, 1, 3, 2],
       [3, 1, 3, 3, 3, 4, 4, 4, 4, 1, 2, 2, 0, 1, 3, 2],
       [4, 2, 4, 2, 2, 5, 5, 4, 3, 2, 1, 1, 1, 0, 3, 3],
       [2, 4, 2, 1, 3, 3, 3, 1, 2, 2, 2, 3, 3, 3, 0, 1],
       [1, 3, 1, 1, 3, 2, 2, 2, 2, 1, 2, 2, 2, 3, 1, 0]])

In [60]:
neighborhood = get_neighborhood_size(apsp)
neighborhood

array([[ 1,  5, 10, 14, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16],
       [ 1,  2,  4,  7, 12, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16],
       [ 1,  3,  7, 13, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16],
       [ 1,  5, 11, 15, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16],
       [ 1,  3,  7, 13, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16],
       [ 1,  2,  5, 10, 14, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16],
       [ 1,  3,  6, 10, 14, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16],
       [ 1,  2,  4,  9, 15, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16],
       [ 1,  4, 10, 14, 15, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16],
       [ 1,  4, 11, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16],
       [ 1,  3,  8, 14, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16],
       [ 1,  4,  8, 13, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16],
       [ 1,  4,  7, 12, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16],
       [ 1,  4,  8, 11, 14, 16, 16, 16, 16, 16, 16,

In [46]:
pivots = generate_maxmin_pivots_new(G.nodes, apsp, 4)
pivots

array([1, 5, 7, 2])

In [48]:
groups = get_pivot_groups_new(G.nodes, apsp, pivots)
groups

{1: array([ 9, 10, 11, 12, 13]),
 5: array([0, 4, 8]),
 7: array([ 3, 14]),
 2: array([ 6, 15])}

In [77]:
groups = get_recursive_pivot_groups(G.nodes, apsp, 4)
groups

{1: {9: array([], dtype=int64),
  10: array([13]),
  11: array([], dtype=int64),
  12: array([], dtype=int64)},
 5: array([0, 4, 8]),
 7: array([ 3, 14]),
 2: array([ 6, 15])}

In [49]:
cardinalities = get_pivot_group_cardinalities(groups)

In [50]:
recur_eset = get_recursive_sparse_edge_set(groups)
# recur_eset

In [52]:
edge_index = create_edge_index(G.edges, recur_eset)
edge_index

tensor([[ 0,  0,  0,  0,  0,  0,  0,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
          1,  1,  1,  1,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
          2,  3,  3,  3,  3,  3,  3,  3,  3,  4,  4,  4,  4,  4,  4,  5,  5,  5,
          5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  6,  6,  6,  6,  6,  7,
          7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  8,  8,  8,  8,
          8,  8,  8,  9,  9,  9,  9,  9,  9,  9, 10, 10, 10, 10, 10, 10, 11, 11,
         11, 11, 11, 11, 11, 12, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13, 13, 13,
         14, 14, 14, 14, 14, 14, 15, 15, 15, 15, 15, 15, 15, 15],
        [ 1,  2,  5,  6,  7,  8, 15,  0,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11,
         12, 13, 14, 15,  0,  1,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14,
         15,  1,  2,  5,  7,  8, 10, 14, 15,  1,  2,  5,  7,  8, 11,  0,  1,  2,
          3,  4,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,  0,  1,  2,  5,  7,  0,
          1,  2,  3,  4,  5,  6,  8,  9, 10

In [82]:
d = apsp[edge_index[0], edge_index[1]]
c = cardinalities[edge_index[0]]
c

array([1, 1, 1, 1, 1, 1, 1, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       1, 1, 1, 1, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1])

In [83]:
n = neighborhood[edge_index[0], d//2]
n

array([10,  5,  1,  1,  5,  1,  1,  4,  4,  4,  4,  4,  4,  4,  4,  2,  2,
        2,  1,  2,  4,  2,  3,  7,  3,  7,  3,  1,  3,  3,  3,  3,  3,  3,
        7,  3,  1, 11,  5,  5,  5,  1,  1,  1,  1,  7,  7,  3,  7,  1,  1,
        1,  5,  2,  2,  2,  2,  5,  2,  2,  5,  5,  5,  5,  2,  2,  1,  6,
        1,  3,  6,  2,  4,  2,  2,  4,  4,  4,  2,  2,  2,  4,  4,  4,  1,
        2,  1, 10,  4,  1,  1,  4,  4,  4,  4,  4,  4,  1,  1,  1,  3,  3,
        1,  8,  3,  1,  4,  4,  1,  8,  8,  1,  1,  1,  4,  7,  7,  1,  1,
        4,  8,  8,  8,  1,  1,  1,  9,  4,  1,  4,  1,  1,  1,  6,  1,  1,
        6,  6,  1,  1])

In [84]:
s = np.minimum(c, n)

In [70]:
generate_symmetric_pivot_edge_attr(edge_index, apsp, cardinalities)

tensor([[1.0000e+00, 5.0000e+00],
        [1.0000e+00, 1.0000e+00],
        [1.0000e+00, 1.0000e+00],
        [1.0000e+00, 1.0000e+00],
        [1.0000e+00, 1.0000e+00],
        [2.0000e+00, 1.0000e+00],
        [3.0000e+00, 4.4444e-01],
        [7.0000e+00, 1.7143e+00],
        [1.0000e+00, 4.0000e+00],
        [1.0000e+00, 4.0000e+00],
        [2.0000e+00, 1.2500e+00],
        [1.0000e+00, 1.0000e+00],
        [1.0000e+00, 1.0000e+00],
        [1.0000e+00, 1.0000e+00],
        [1.0000e+00, 1.0000e+00],
        [1.0000e+00, 5.0000e+00],
        [2.0000e+00, 1.2500e+00],
        [1.0000e+00, 2.0000e+01],
        [5.0000e+00, 4.8000e+00],
        [1.0000e+00, 5.0000e+00],
        [2.0000e+00, 1.2500e+00],
        [1.0000e+00, 4.0000e+00],
        [1.0000e+00, 1.0000e+00],
        [2.0000e+00, 1.0000e+00],
        [1.0000e+00, 1.0000e+00],
        [2.0000e+00, 1.2500e+00],
        [3.0000e+00, 5.5556e-01],
        [3.0000e+00, 5.5556e-01],
        [1.0000e+00, 1.2000e+02],
        [1.000

In [37]:
[(i, j) for i in range(5) for j in range(i)]

[(1, 0),
 (2, 0),
 (2, 1),
 (3, 0),
 (3, 1),
 (3, 2),
 (4, 0),
 (4, 1),
 (4, 2),
 (4, 3)]

In [51]:
np.array([[1,2,3],[4,5,6],[7,8,9]])[[0,2], :][:, [0,2]]

array([[1, 3],
       [7, 9]])

In [77]:
np.append(a, 1)

array([1.])

In [21]:
np.argmax(list(map(nx.classes.graph.Graph.number_of_nodes, G_list)))

8473

In [17]:
type(G)

networkx.classes.graph.Graph